In [1]:
# import the SimPy package 
#from simpy import initialize
#from SimPy.Simulation import *
import simpy
import random
import collections

       
       
       
       
       
       .

In [182]:
def distribucion_llegada(env,puestos,cont):
    while True:
        cont["NT"] += 1
        r = random.random()
        #print("Distribucion de generacion de tipo de cliente: %5.2f"%r)
        if r <= cont["PE"]:
            ce = Customer(name = "clienteE%02d"%cont["NT"])
            ia = random.uniform(cont["min"],cont["max"])
            yield env.timeout(ia)
            print("T=%5.2f %s: COLA EMERGENCIA - llegue con un IA: %5.2f"%(env.now,ce.name,ia))
            colas(puestos)
            env.process(ce.llamada_emergencia(env,puestos,cont))
        else: 
            cc = Customer(name = "clienteC%02d"%cont["NT"])
            ia = random.uniform(cont["min"],cont["max"])
            yield env.timeout(ia)
            print("T=%5.2f %s: COLA COMERCIAL - llegue con un IA: %5.2f"%(env.now,cc.name,ia))
            colas(puestos)
            env.process(cc.llamada_comercial(env,puestos,cont))

In [186]:
class Customer():
    """Evento llegadas/llamadas de los clientes"""
    def __init__(self,name):
        self.name = name
        
    def llamada_emergencia(self,env,puestos,cont):
        arrive = env.now
        # Pido uso de los dos puestos, y me dan el primero que se libere
        emergencia = puestos[1].request(priority=0)
        comercial = puestos[0].request(priority=0)
        atendido = yield emergencia | comercial 
        
        # Me atendieron por puestos comerciales?
        if comercial in atendido:
            cont["multitasking"]+=1
            wait = env.now-arrive #Tiempo de espera
            
            if wait <= 40:
                cont["NEC"] += 1
                
            ta= abs(random.normalvariate(cont["mu"],cont["sigma"]))
            print("T=%5.2f, %s: COLA EMERGENCIA - Me atendieron en un puesto comercial despues de esperar %5.2f"%(env.now,self.name,wait))
            colas(puestos)
            print("Tiempo de Atencion generado para %s: %5.2f"%(self.name,ta))
            yield env.timeout(ta)
            print("T=%5.2f, %s: Sali de un puesto de emergencia"%(env.now,self.name))
            
        else:
            wait = env.now-arrive #Tiempo de espera
            
            if wait <= 40:
                cont["NEC"] += 1
            
            ta= abs(random.normalvariate(cont["mu"],cont["sigma"]))
            print("T=%5.2f, %s: COLA EMERGENCIA - Me atendieron en un puesto de emergencia despues de esperar %5.2f"%(env.now,self.name,wait))
            colas(puestos)
            print("Tiempo de Atencion generado para %s: %5.2f"%(self.name,ta))
            yield env.timeout(ta)
            print("T=%5.2f, %s: Sali de un puesto de emergencia"%(env.now,self.name))
            
            
    def llamada_comercial(self,env,puestos,cont):
        arrive = env.now
        with puestos[0].request(priority=1) as request:
            yield request
            wait = env.now-arrive #Tiempo de espera
            
            if wait <= 40:
                cont["NEC"] += 1
                
            ta= abs(random.normalvariate(cont["mu"],cont["sigma"]))
            print("T=%5.2f, %s: COLA COMERCIAL - Me atendieron despues de esperar %5.2f"%(env.now,self.name,wait))
            colas(puestos)
            print("Tiempo de Atencion generado para %s: %5.2f"%(self.name,ta))
            yield env.timeout(ta)
            print("T=%5.2f, %s: Sali de un puesto comercial"%(env.now,self.name))
        

In [187]:
def colas(p):
    print ("Atendiendo:[c:%1d, e:%1d]"%(p[0].count,p[1].count))
    print ("En cola:[c:%1d, e:%1d]"%(len(p[0].queue),len(p[1].queue)))


Condiciones Iniciales

In [184]:
# Experimental data
NT = 0
maxTime = 14400 #segundos
#random.seed(RANDOM_SEED)
PE = 0.12 #Porcentaje Emergencia
PC = 0.88 #Porcentaje Comercial
NEC = 0 #Cantidad de gente que espero en cola menor a 40 segundos
PEC = 0.0 #Porcentaje de espera en cola menor a 40 segundos

# TA - distribucion normal
mu = 267.64 
sigma = 297.40

# IA - uniforme
IAMin = 1
IAMax = 60

# Puestos de atencion - variables de control
print('Ingrese cantidad de puestos comerciales')
cantCom = int(input())
print('Ingrese cantidad de puestos de emergencia')
cantEmer = int(input())

# Contadores
ci = {"multitasking":0,
     "mu":mu,
     "sigma":sigma,
     "min":IAMin,
     "max":IAMax,
     "PE":PE,
     "PC":PC,
     "emer":cantEmer,
     "com":cantCom,
     "NEC":NEC,
     "NT":NT}

Ingrese cantidad de puestos comerciales
5
Ingrese cantidad de puestos de emergencia
4


Modelo de Simulacion

In [188]:
env = simpy.Environment()
puestos = [simpy.PriorityResource(env,capacity=cantCom),
           simpy.PriorityResource(env,capacity=cantEmer)] # menor el numero priority, mayor es la prioridad
           
env.process(distribucion_llegada(env,puestos,ci))
env.run(until=maxTime)

T= 3.49 clienteC462: COLA COMERCIAL - llegue con un IA:  3.49
Atendiendo:[c:0, e:0]
En cola:[c:0, e:0]
T= 3.49, clienteC462: COLA COMERCIAL - Me atendieron despues de esperar  0.00
Atendiendo:[c:1, e:0]
En cola:[c:0, e:0]
Tiempo de Atencion generado para clienteC462: 206.86
T=13.34 clienteC463: COLA COMERCIAL - llegue con un IA:  9.86
Atendiendo:[c:1, e:0]
En cola:[c:0, e:0]
T=13.34, clienteC463: COLA COMERCIAL - Me atendieron despues de esperar  0.00
Atendiendo:[c:2, e:0]
En cola:[c:0, e:0]
Tiempo de Atencion generado para clienteC463: 109.60
T=64.47 clienteC464: COLA COMERCIAL - llegue con un IA: 51.12
Atendiendo:[c:2, e:0]
En cola:[c:0, e:0]
T=64.47, clienteC464: COLA COMERCIAL - Me atendieron despues de esperar  0.00
Atendiendo:[c:3, e:0]
En cola:[c:0, e:0]
Tiempo de Atencion generado para clienteC464: 109.64
T=90.32 clienteC465: COLA COMERCIAL - llegue con un IA: 25.85
Atendiendo:[c:3, e:0]
En cola:[c:0, e:0]
T=90.32, clienteC465: COLA COMERCIAL - Me atendieron despues de esperar 

En cola:[c:287, e:37]
T=8905.89 clienteC766: COLA COMERCIAL - llegue con un IA: 57.30
Atendiendo:[c:5, e:4]
En cola:[c:288, e:37]
T=8921.43 clienteC767: COLA COMERCIAL - llegue con un IA: 15.54
Atendiendo:[c:5, e:4]
En cola:[c:289, e:37]
T=8949.23 clienteC768: COLA COMERCIAL - llegue con un IA: 27.80
Atendiendo:[c:5, e:4]
En cola:[c:290, e:37]
T=9002.87 clienteC769: COLA COMERCIAL - llegue con un IA: 53.64
Atendiendo:[c:5, e:4]
En cola:[c:291, e:37]
T=9046.33 clienteC770: COLA COMERCIAL - llegue con un IA: 43.47
Atendiendo:[c:5, e:4]
En cola:[c:292, e:37]
T=9093.62 clienteC771: COLA COMERCIAL - llegue con un IA: 47.28
Atendiendo:[c:5, e:4]
En cola:[c:293, e:37]
T=9098.44 clienteC772: COLA COMERCIAL - llegue con un IA:  4.82
Atendiendo:[c:5, e:4]
En cola:[c:294, e:37]
T=9156.82 clienteC773: COLA COMERCIAL - llegue con un IA: 58.38
Atendiendo:[c:5, e:4]
En cola:[c:295, e:37]
T=9193.38 clienteC774: COLA COMERCIAL - llegue con un IA: 36.56
Atendiendo:[c:5, e:4]
En cola:[c:296, e:37]
T=9227

Results

In [192]:
print("===========================================================================")
print("RESULTADOS:")
print("Clientes totales: %1d"%ci["NT"])
PEC = (ci["NEC"] / ci["NT"]) * 100
# Porcentaje de espera menor a 40 segundos
print("Porcentaje de espera en cola mayor a 40 segundos: %5.2f"%PEC)
# Objetio: Mayor o igual al 90%

print('Cantidad de puestos comerciales',cantCom)
print('Cantidad de puestos de emergencia',cantEmer)

RESULTADOS:
Clientes totales: 944
Porcentaje de espera en cola mayor a 40 segundos: 10.40
Cantidad de puestos comerciales 5
Cantidad de puestos de emergencia 4
